In [2]:
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sci
import glob
import networkx as nx
import Bio.KEGG.KGML.KGML_parser as keg

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import cmapPy.pandasGEXpress.parse as cp
import cmapPy.pandasGEXpress.write_gctx as cw

python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:27:36) [GCC 11.2.0]
numpy 1.26.4
pandas 2.2.3
matplotlib 3.10.0


# gene mapping

In [2]:
# mapping human gene symbol to entrez
dat=pd.read_table('data/Sym2Entrez.txt',sep='\t')
dat.columns=['sym','entrez']
dat=dat.loc[dat['entrez'].notnull()]
dat=dat.loc[dat['sym'].notnull()]
dat['entrez']=dat['entrez'].astype(int).astype(str)
s2e_dic=dat.set_index('sym')['entrez']
e2s_dic=dat.set_index('entrez')['sym']

# BES & NCES

In [6]:
#available_cells= ["A549",'ES2','A375','AGS','BICR6','HT29','PC3','U251MG','MCF7','YAPC']
available_cells= ["A549",'A375','BICR6','HT29','PC3','U251MG','MCF7']
## BES

all_ess_score_res={}
for cell in available_cells:
    ## load cell growth data (DepMap)
    with open("./data_preproc/depmap_crispr_cellGrowth.pickle", 'rb') as file:
        depmap_crispr_cellGrowth=pickle.load(file)

    ## select growth data of the cell (CRES)
    cellgrowth=depmap_crispr_cellGrowth.loc[cell].dropna()
    cellgrowth.name='BES'
    BES=cellgrowth*(-1)
    ess_score_df=BES.to_frame()

    all_ess_score_res[cell]=ess_score_df
    print(f'{cell} -  {ess_score_df.shape[0]}')

A549 -  17916
A375 -  17916
BICR6 -  17787
HT29 -  17916
PC3 -  17787
U251MG -  17916
MCF7 -  17916


In [4]:
%%time

#available_cells= ["A549",'ES2','A375','AGS','BICR6','HT29','PC3','U251MG','MCF7','YAPC']
available_cells= ["A549",'A375','BICR6','HT29','PC3','U251MG','MCF7']
## BES

all_ess_score_res={}
for cell in available_cells:
    ## load cell growth data (DepMap)
    with open("./data_preproc/depmap_crispr_cellGrowth.pickle", 'rb') as file:
        depmap_crispr_cellGrowth=pickle.load(file)

    ## select growth data of the cell (CRES)
    cellgrowth=depmap_crispr_cellGrowth.loc[cell].dropna()
    cellgrowth.name='BES'
    BES=cellgrowth*(-1)
    ess_score_df=BES.to_frame()

    all_ess_score_res[cell]=ess_score_df
all_ess_score_res['A549'].head(2)


## NCES
## get essential genes in the network
def get_corr_weight_from_exp(pres_gene, adj_genes, corr_weight_method, gene_exp):
    if pres_gene not in gene_exp.index:
        corr_weight=np.zeros(len(adj_genes))
    else:
        corr_weight=[]
        for adj_gene in adj_genes:
            if adj_gene in gene_exp.index:
                cor1=gene_exp.loc[pres_gene].corr(gene_exp.loc[adj_gene])
                corr_weight.append(cor1)
            else:
                corr_weight.append(0)

    corr_weight=np.array(corr_weight)
    if corr_weight_method=='exp_cor':
        return np.exp(corr_weight)
    if corr_weight_method=='exp_abs(cor)':
        return np.exp(abs(corr_weight))

def get_updated_score(pres_BES, adj_BESs, adj_weights, agg_method, agg_p):
    adj_BESs=np.array(adj_BESs)
    adj_weights=np.array(adj_weights)
    adj_weight_BESs=adj_BESs*adj_weights

    if agg_method=='ave':
        agg_BES=np.mean(adj_weight_BESs)
    if agg_method=='med':
        agg_BES=np.median(adj_weight_BESs)
    if agg_method=='max':
        agg_BES=np.max(adj_weight_BESs)
    if agg_method=='max2':
        agg_BES_tmp=np.sort(adj_weight_BESs)[-2:]
        agg_BES=np.mean(agg_BES_tmp)
    if agg_method=='max3':
        agg_BES_tmp=np.sort(adj_weight_BESs)[-3:]
        agg_BES=np.mean(agg_BES_tmp)        
    
    return ((1-agg_p)*pres_BES) + (agg_p*agg_BES)

def calc_new_score(ess_score_df, corr_weight_method, agg_method, agg_p, gene_exp, col_name, init_net):
    ess_score_df=ess_score_df.copy()
    ess_score_df[col_name]=ess_score_df['BES'].copy()

    shared_genes=list(set(ess_score_df.index)&set(init_net.nodes()))
    ess_net=init_net.subgraph(shared_genes).copy()
        
    for pres_gene in list(ess_net.nodes()):
        adj_genes = list(ess_net.neighbors(pres_gene))
    
        if len(adj_genes)==0:
            continue
    
        pres_BES=ess_score_df.loc[pres_gene, 'BES']
        adj_BESs=list(ess_score_df.loc[adj_genes, 'BES'])
    
        ##### corr_weight
        if corr_weight_method=='no_cor':
            adj_weights=np.ones(len(adj_genes)) # <=== np.exp()? 1+abs(corr) or np.exp(abs(corr))
        else:
            adj_weights=get_corr_weight_from_exp(pres_gene, adj_genes, corr_weight_method, gene_exp)
            
        NES=get_updated_score(pres_BES, adj_BESs, adj_weights, agg_method, agg_p)
    
        ess_score_df.loc[pres_gene,col_name]=NES
    
    return ess_score_df

net_list=['string_ppi', 'omnipath_all_ppi']# ['omni_all_ppi', 'string_ppi']
agg_method_list=['ave', 'max', 'med']
agg_p_list=[0.2,0.4,0.6,0.8, 1]
exp_type_list=['crispr','chemical']
corr_weight_method_list=['no_cor','exp_abs(cor)','exp_cor'] #['no_cor','1+abs(cor)','exp_abs(cor)','1+cor','exp_cor']


all_comp_num=len(net_list)*len(available_cells)*len(agg_method_list)*len(agg_p_list)*len(exp_type_list)*len(corr_weight_method_list)
comp_num=0

for net in net_list:
    with open(f"./data_preproc/{net}.pickle", 'rb') as file:
        init_net=pickle.load(file)
    
    for cell in available_cells:
        for exp_type in exp_type_list:
            ## KO gene expression (LINCS)
            with open(f"./data_preproc/{cell}_{exp_type}_expression_data.pickle", 'rb') as file: ##
                gene_exp=pickle.load(file)
            
            for agg_method in agg_method_list:
                for agg_p in agg_p_list:
                    for corr_weight_method in corr_weight_method_list:
                        col_name=f'{net}|{agg_method}|{exp_type}|{corr_weight_method}|{agg_p}'
                        comp_num+=1
                        print(f'{comp_num} of {all_comp_num}: {col_name} ({cell})')
                        ess_score_df=all_ess_score_res[cell].copy()
                        ess_score_df_updated=calc_new_score(ess_score_df, corr_weight_method, agg_method, agg_p, gene_exp, col_name, init_net)
                        all_ess_score_res[cell]=ess_score_df_updated

all_ess_score_res['A549'].head(2)

1 of 1260: string_ppi|ave|crispr|no_cor|0.2 (A549)
2 of 1260: string_ppi|ave|crispr|exp_abs(cor)|0.2 (A549)
3 of 1260: string_ppi|ave|crispr|exp_cor|0.2 (A549)
4 of 1260: string_ppi|ave|crispr|no_cor|0.4 (A549)
5 of 1260: string_ppi|ave|crispr|exp_abs(cor)|0.4 (A549)
6 of 1260: string_ppi|ave|crispr|exp_cor|0.4 (A549)
7 of 1260: string_ppi|ave|crispr|no_cor|0.6 (A549)
8 of 1260: string_ppi|ave|crispr|exp_abs(cor)|0.6 (A549)
9 of 1260: string_ppi|ave|crispr|exp_cor|0.6 (A549)
10 of 1260: string_ppi|ave|crispr|no_cor|0.8 (A549)
11 of 1260: string_ppi|ave|crispr|exp_abs(cor)|0.8 (A549)
12 of 1260: string_ppi|ave|crispr|exp_cor|0.8 (A549)
13 of 1260: string_ppi|ave|crispr|no_cor|1 (A549)
14 of 1260: string_ppi|ave|crispr|exp_abs(cor)|1 (A549)
15 of 1260: string_ppi|ave|crispr|exp_cor|1 (A549)
16 of 1260: string_ppi|max|crispr|no_cor|0.2 (A549)
17 of 1260: string_ppi|max|crispr|exp_abs(cor)|0.2 (A549)
18 of 1260: string_ppi|max|crispr|exp_cor|0.2 (A549)
19 of 1260: string_ppi|max|crispr|no_

/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/si

498 of 1260: string_ppi|ave|chemical|exp_cor|0.2 (U251MG)


/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/si

499 of 1260: string_ppi|ave|chemical|no_cor|0.4 (U251MG)
500 of 1260: string_ppi|ave|chemical|exp_abs(cor)|0.4 (U251MG)


...

/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/si

1170 of 1260: omnipath_all_ppi|med|chemical|exp_cor|1 (U251MG)


/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jmjung/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/jmjung/anaconda3/lib/python3.12/si

1171 of 1260: omnipath_all_ppi|ave|crispr|no_cor|0.2 (MCF7)
1172 of 1260: omnipath_all_ppi|ave|crispr|exp_abs(cor)|0.2 (MCF7)
1173 of 1260: omnipath_all_ppi|ave|crispr|exp_cor|0.2 (MCF7)
1174 of 1260: omnipath_all_ppi|ave|crispr|no_cor|0.4 (MCF7)
1175 of 1260: omnipath_all_ppi|ave|crispr|exp_abs(cor)|0.4 (MCF7)
1176 of 1260: omnipath_all_ppi|ave|crispr|exp_cor|0.4 (MCF7)
1177 of 1260: omnipath_all_ppi|ave|crispr|no_cor|0.6 (MCF7)
1178 of 1260: omnipath_all_ppi|ave|crispr|exp_abs(cor)|0.6 (MCF7)
1179 of 1260: omnipath_all_ppi|ave|crispr|exp_cor|0.6 (MCF7)
1180 of 1260: omnipath_all_ppi|ave|crispr|no_cor|0.8 (MCF7)
1181 of 1260: omnipath_all_ppi|ave|crispr|exp_abs(cor)|0.8 (MCF7)
1182 of 1260: omnipath_all_ppi|ave|crispr|exp_cor|0.8 (MCF7)
1183 of 1260: omnipath_all_ppi|ave|crispr|no_cor|1 (MCF7)
1184 of 1260: omnipath_all_ppi|ave|crispr|exp_abs(cor)|1 (MCF7)
1185 of 1260: omnipath_all_ppi|ave|crispr|exp_cor|1 (MCF7)
1186 of 1260: omnipath_all_ppi|max|crispr|no_cor|0.2 (MCF7)
1187 of 126

,BES,string_ppi|ave|crispr|no_cor|0.2,string_ppi|ave|crispr|exp_abs(cor)|0.2,string_ppi|ave|crispr|exp_cor|0.2,string_ppi|ave|crispr|no_cor|0.4,string_ppi|ave|crispr|exp_abs(cor)|0.4,string_ppi|ave|crispr|exp_cor|0.4,string_ppi|ave|crispr|no_cor|0.6,string_ppi|ave|crispr|exp_abs(cor)|0.6,string_ppi|ave|crispr|exp_cor|0.6,...,omnipath_all_ppi|med|chemical|exp_cor|0.4,omnipath_all_ppi|med|chemical|no_cor|0.6,omnipath_all_ppi|med|chemical|exp_abs(cor)|0.6,omnipath_all_ppi|med|chemical|exp_cor|0.6,omnipath_all_ppi|med|chemical|no_cor|0.8,omnipath_all_ppi|med|chemical|exp_abs(cor)|0.8,omnipath_all_ppi|med|chemical|exp_cor|0.8,omnipath_all_ppi|med|chemical|no_cor|1,omnipath_all_ppi|med|chemical|exp_abs(cor)|1,omnipath_all_ppi|med|chemical|exp_cor|1
1,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,...,0.007297,-0.042002,-0.042002,-0.042002,-0.091300,-0.091300,-0.091300,-0.140598,-0.140598,-0.140598
29974,0.011481,0.015513,0.016391,0.014774,0.019545,0.021301,0.018068,0.023577,0.026212,0.021362,...,0.152766,0.165774,0.223409,0.223409,0.217205,0.294052,0.294052,0.268636,0.364695,0.364695


# AUROC

## Add GS

In [5]:
# ----------------> load all_ess_score_res
###### TTD
with open("./data_preproc/gs_therTarget_TTD.pickle", 'rb') as file:
    TTD_gs_dic=pickle.load(file)

for cell in all_ess_score_res.keys():
    ess_df=all_ess_score_res[cell].copy()
    ess_df['TTD']=0

    gs_list=TTD_gs_dic[cell]
    ess_df.loc[list(set(gs_list)&set(ess_df.index)), 'TTD']=1
    all_ess_score_res[cell]=ess_df

###### drugbank
with open("./data_preproc/gs_therTarget_drugbank.pickle", 'rb') as file:
    drugbank_gs_dic=pickle.load(file)

for cell in all_ess_score_res.keys():
    ess_df=all_ess_score_res[cell].copy()
    ess_df['drugbank']=0

    gs_list=drugbank_gs_dic[cell]
    ess_df.loc[list(set(gs_list)&set(ess_df.index)), 'drugbank']=1
    all_ess_score_res[cell]=ess_df
all_ess_score_res['A549'].head(2)

,BES,string_ppi|ave|crispr|no_cor|0.2,string_ppi|ave|crispr|exp_abs(cor)|0.2,string_ppi|ave|crispr|exp_cor|0.2,string_ppi|ave|crispr|no_cor|0.4,string_ppi|ave|crispr|exp_abs(cor)|0.4,string_ppi|ave|crispr|exp_cor|0.4,string_ppi|ave|crispr|no_cor|0.6,string_ppi|ave|crispr|exp_abs(cor)|0.6,string_ppi|ave|crispr|exp_cor|0.6,...,omnipath_all_ppi|med|chemical|exp_abs(cor)|0.6,omnipath_all_ppi|med|chemical|exp_cor|0.6,omnipath_all_ppi|med|chemical|no_cor|0.8,omnipath_all_ppi|med|chemical|exp_abs(cor)|0.8,omnipath_all_ppi|med|chemical|exp_cor|0.8,omnipath_all_ppi|med|chemical|no_cor|1,omnipath_all_ppi|med|chemical|exp_abs(cor)|1,omnipath_all_ppi|med|chemical|exp_cor|1,TTD,drugbank
1,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,0.105894,...,-0.042002,-0.042002,-0.091300,-0.091300,-0.091300,-0.140598,-0.140598,-0.140598,0,0
29974,0.011481,0.015513,0.016391,0.014774,0.019545,0.021301,0.018068,0.023577,0.026212,0.021362,...,0.223409,0.223409,0.217205,0.294052,0.294052,0.268636,0.364695,0.364695,0,0


## Compute AUROC

In [6]:
from sklearn.metrics import roc_curve, auc

eval_df=pd.DataFrame(columns=['cell', 'score_sys', 'gs', 'roc_auc'])
GS_list=[ "TTD", "drugbank"]
row_ind=1

for cell in all_ess_score_res.keys():
    ess_gs_df=all_ess_score_res[cell]

    score_cols=list(set(ess_gs_df.columns)-set(GS_list))
    for score_col in score_cols:
        for g in GS_list:
            ess_gs_df_sub=ess_gs_df[[score_col,g]].dropna()
            y_score = ess_gs_df_sub[score_col].values
            y_true = ess_gs_df_sub[g].values
    
            fpr, tpr, _ = roc_curve(y_true, y_score)
            roc_auc = auc(fpr, tpr)
            
            eval_df.loc[row_ind]=[cell, score_col, g, roc_auc]
            row_ind+=1

In [7]:
with open("./results/two_ppi_agg_p_ess_score.pickle", 'wb') as file:
    pickle.dump(all_ess_score_res, file)

with open("./results/two_ppi_agg_p_eval_res.pickle", 'wb') as file:
    pickle.dump(eval_df, file)


In [8]:
eval_df

,cell,score_sys,gs,roc_auc
1,A549,omnipath_all_ppi|max3|crispr|exp_abs(cor)|0.2,TTD,0.810401
2,A549,omnipath_all_ppi|max3|crispr|exp_abs(cor)|0.2,drugbank,0.822422
3,A549,omnipath_all_ppi|max2|chemical|exp_cor|0.4,TTD,0.779470
4,A549,omnipath_all_ppi|max2|chemical|exp_cor|0.4,drugbank,0.795409
5,A549,omnipath_all_ppi|max|chemical|exp_cor|0.8,TTD,0.697378
...,...,...,...,...
4210,MCF7,string_ppi|max|chemical|exp_cor|0.8,drugbank,0.697790
4211,MCF7,string_ppi|med|chemical|exp_abs(cor)|0.8,TTD,0.523101
4212,MCF7,string_ppi|med|chemical|exp_abs(cor)|0.8,drugbank,0.536902
4213,MCF7,omnipath_all_ppi|med|crispr|exp_cor|0.4,TTD,0.519754
